In [3]:
import os
%pwd

'd:\\MV\\SWCODES\\PYTHON_RELATED\\MLOPS\\End-to-End-ML-project-with-ML-flow\\research'

In [6]:
os.chdir('../')

In [7]:
pwd

'd:\\MV\\SWCODES\\PYTHON_RELATED\\MLOPS\\End-to-End-ML-project-with-ML-flow'

In [64]:
import pandas as pd


In [14]:
d = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

d.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [15]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [21]:
d.isna()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0             False             False        False           False      False   
1             False             False        False           False      False   
2             False             False        False           False      False   
3             False             False        False           False      False   
4             False             False        False           False      False   
...             ...               ...          ...             ...        ...   
1594          False             False        False           False      False   
1595          False             False        False           False      False   
1596          False             False        False           False      False   
1597          False             False        False           False      False   
1598          False             False        False           False      False   

      free sulfur dioxide  total sulfur dioxide  density     pH  sulphates  \
0                   False                 False    False  False      False   
1                   False                 False    False  False      False   
2                   False                 False    False  False      False   
3                   False                 False    False  False      False   
4                   False                 False    False  False      False   
...                   ...                   ...      ...    ...        ...   
1594                False                 False    False  False      False   
1595                False                 False    False  False      False   
1596                False                 False    False  False      False   
1597                False                 False    False  False      False   
1598                False                 False    False  False      False   

      alcohol  quality  
0       False    False  
1       False    False  
2       False    False  
3       False    False  
4       False    False  
...       ...      ...  
1594    False    False  
1595    False    False  
1596    False    False  
1597    False    False  
1598    False    False  

[1599 rows x 12 columns]

In [25]:
d.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [26]:
# if have complex data, perform ETL operations to clean data

In [30]:
d.shape

(1599, 12)

In [35]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE:str
    unzip_data_dir : Path
    all_schema : dict

In [36]:
from MLproject.constants import *
from MLproject.utils.common import read_yaml,create_directories
 

In [60]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
      config = self.config.data_validation
      schema  = self.schema.COLUMNS

      create_directories([config.root_dir])

      data_validation_config = DataValidationConfig(
      root_dir=config.root_dir,
      STATUS_FILE=config.STATUS_FILE,
      unzip_data_dir=config.unzip_dir,
      all_schema=schema,
      )


      return data_validation_config

In [61]:
import os
from MLproject import logger

In [67]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status=None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status= False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation status:{validation_status}")
                else:
                    validation_status= True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation status:{validation_status}")
            return validation_status
        except Exception as e:
            raise e





In [66]:
try:
    config = ConfigurationManager()
    data_validation_config  = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    
except Exception as e:
    raise e

[2025-05-20 00:24:22,631: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-20 00:24:22,639: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-20 00:24:22,639: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-20 00:24:22,646: INFO: common: created directory at: artifacts]
[2025-05-20 00:24:22,646: INFO: common: created directory at: artifacts/data_validation]


D:\MV\SWCODES\PYTHON_RELATED\MLOPS\End-to-End-ML-project-with-ML-flow\artifacts\data_ingestion\winequality-red.csv